In [1]:
# Notebook for Testing

In [13]:
from music21 import converter, instrument, note, chord, stream
from mido import MidiFile
import numpy
import glob
import pickle
import numpy as np

In [15]:
import pygame

pygame.init()
pygame.mixer.music.load(r'midi\kaggle\mozart\mz_311_1.mid')
pygame.mixer.music.play()

while pygame.mixer.music.get_busy():
    pygame.time.wait(1000)

In [11]:

# def get_notes():
#     """ Get all the notes and chords from the midi files in the ./midi_songs directory """
#     notes = []

#     for file in glob.glob("midi/RondoAllaTurca.mid"):
#         midi = converter.parse(file)
#         print(midi.isStream)

#         print("Parsing %s" % file)

#         notes_to_parse = None

#         try: # file has instrument parts
#             s2 = instrument.partitionByInstrument(midi)
#             sp = instrument.Piano(midi)
#             notes_to_parse = s2.parts[0].recurse()

#         except: # file has notes in a flat structure
#             notes_to_parse = midi.flat.notes
        # print("FLAT\n", notes_to_parse)

def get_notes(filename):
    buffer_size = 500000
    notes = [''] * buffer_size
    Last_time = 0
    offset = []
    midi = converter.parse(filename)
    
    print('parsing {}'.format(filename))
    try:  # file has instrument parts
        s2 = instrument.partitionByInstrument(midi)
        notes_to_parse = s2.recurse()
    except:  # file has notes in a flat structure
        notes_to_parse = midi.flat.notes

    for element in notes_to_parse:
      if isinstance(element, note.Note):
        
         time = int(element.offset * 4)
        # print(element)
         if notes[time] == '':
            notes[time] = str(element.pitch.midi)
            last_time = time
      elif isinstance(element, chord.Chord):
         time = int(element.offset * 4)
         if notes[time] == '':
            notes[time] = '.'.join(str(n) for n in element.normalOrder)
            last_time = time

    
    return notes[:last_time]

    #     for element in notes_to_parse:
            
    #         if isinstance(element, note.Note):
                
    #             notes.append(str(element.pitch))
    #         elif isinstance(element, chord.Chord):
    #             print(element, '.'.join(str(n) for n in element.normalOrder))
    #             notes.append('.'.join(str(n) for n in element.normalOrder))

    # with open('data/notes', 'wb') as filepath:
    #     pickle.dump(notes, filepath)

    # return notes

notes = get_notes("midi\kaggle\mozart\mz_311_1.mid")
notes[-100::]

SubConverterException: Cannot find a path to the 'mscore' file at c:\Users\mohit\OneDrive\Desktop\Programming\Open_Source_Projects\Markov-Music\C:\Program Files\MuseScore 3\MuseScore.exe -- download MuseScore

In [5]:
def prepare_sequences(notes, n_vocab, note_to_int):
   sequence_length = 400
   network_input = []
   network_output = []
   for i in range(0, len(notes) - sequence_length, 1):
      sequence_in = notes[i:i + sequence_length]
      sequence_out = notes[i + sequence_length]
      network_input.append([note_to_int[char] for char in sequence_in])
      network_output.append(note_to_int[sequence_out])
   n_patterns = len(network_input)
   normalized_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
   normalized_input = normalized_input / float(n_vocab)
   # network_output = to_categorical(network_output)
   return (network_input, normalized_input, network_output)

In [6]:
pitchnames = sorted(set(item for item in notes))  
# create a dictionary to map pitches to integers
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
int_to_note = dict((note, number) for number, note in enumerate(pitchnames))

network_input, normalized_input, network_output = prepare_sequences(notes, len(set(notes)), note_to_int)

In [7]:
# max(max(network_input))
network_input[40:80]

[[67,
  62,
  61,
  62,
  67,
  0,
  62,
  0,
  60,
  0,
  61,
  0,
  0,
  0,
  0,
  0,
  62,
  61,
  59,
  61,
  67,
  62,
  61,
  62,
  67,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  67,
  0,
  69,
  0,
  71,
  0,
  72,
  0,
  67,
  0,
  43,
  0,
  61,
  0,
  43,
  0,
  67,
  62,
  61,
  62,
  67,
  62,
  61,
  59,
  57,
  0,
  47,
  0,
  50,
  0,
  47,
  0,
  38,
  38,
  47,
  0,
  38,
  61,
  59,
  57,
  56,
  0,
  48,
  0,
  50,
  0,
  48,
  0,
  59,
  47,
  48,
  47,
  45,
  43,
  42,
  36,
  59,
  0,
  42,
  0,
  50,
  0,
  42,
  0,
  53,
  53,
  42,
  0,
  53,
  62,
  61,
  59,
  57,
  0,
  43,
  0,
  50,
  0,
  43,
  0,
  61,
  30,
  31,
  32,
  33,
  35,
  36,
  42,
  61,
  57,
  59,
  61,
  62,
  67,
  69,
  71,
  72,
  71,
  69,
  67,
  62,
  61,
  59,
  57,
  59,
  61,
  59,
  58,
  59,
  61,
  62,
  67,
  69,
  67,
  62,
  61,
  59,
  57,
  56,
  54,
  56,
  57,
  56,
  50,
  57,
  0,
  30,
  0,
  62,
  67,
  62,
  59,
  62,
  61,
  59,
  57,
  56,
  57,
  56,
  50,
  57,
  0,


In [ ]:
def create_midi(prediction_output):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """

    output_notes = []

    # create note and chord objects based on the values generated by the model
    for idx, pattern in enumerate(prediction_output):
      if len(pattern) != 0:
        notes_in_chord = [int(n) for n in pattern.split('.')]
        new_chord = chord.Chord(notes_in_chord)
        new_chord.offset = idx/4
        output_notes.append(new_chord)
        # increase offset each iteration so that notes do not stack

    midi_stream = stream.Stream(output_notes)

    midi_stream.write('midi', fp='test_output.mid')